In [1]:
import numpy as np
from gameimage import board_to_image, update_image
import chess
from game import Game
from actionspace import map_w, map_b
import tensorflow as tf
from tf_funcs import predict_fn
from train.config import TrainingConfig
from math import log
from time import time
from trt_funcs import load_trt_model_latest
import pympler.asizeof
trt_func, model = load_trt_model_latest()


2024-03-26 13:39:33.730974: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-26 13:39:35.966123: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5909 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2070 SUPER, pci bus id: 0000:26:00.0, compute capability: 7.5
2024-03-26 13:39:36.684918: E tensorflow/compiler/tf2tensorrt/utils/trt_logger.cc:87] DefaultLogger 3: [runtime.cpp::~Runtime::346] Error Code 3: API Usage Error (Parameter check failed at: runtime/rt/runtime.cpp::~Runtime::346, condition: mEngineCounter.use_count() == 1. Destroying a runtime before destroying deserialized engines created by the runtime leads to undefined behavior.
)


In [2]:
from mcts import run_mcts
from test_games import ruy_lopez, scholars_mate, single_move
from time import time
config = TrainingConfig()
game = Game(single_move)

In [3]:
move, root, visit_counts = run_mcts(game, config, trt_func, 800, time_limit=5.0, root=None, verbose_move=True, engine_play=True)

Time for MCTS: 3.63s
Visits: 0
Summed value: 0.000000
Evaluation: 0.503397
Best variation: e8f7
Move          Visits  NN Output                    Policy             Prior          Avg. value     UCB            Q+U           
----------------------------------------------------------------------------------------------------------------------------------
e8f7  (None)  N: 1    (v: 0.000000, p:  0.188337)  (p_norm: 100.00%)  (P: 1.000000)  (Q: 0.000000)  (U: 0.000000)  (Q+U: 0.000000)


In [4]:
from test_games import ruy_lopez_cpp, scholars_mate_cpp, single_move_cpp
from mcts.c import run_mcts as run_mcts_c
game = Game(single_move_cpp)
move, root, visit_counts = run_mcts_c(game, config, trt_func, 800, time_limit=15.0, verbose_move=True)
root.describe()

e8f7 -> P: 1.000000, Q: 0.000000, W: 0.000000, N: 1
